# TRAINING CODE

In [6]:
  
%reload_ext tensorboard
%tensorboard --logdir="C:/Users/bethg/Downloads/Desktop_prev/Untitled Folder/tensorboard20240219-153331"



In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import datetime
import os

# Load annotations from CSV files
train_annotations = pd.read_csv(r'C:\Users\bethg\Downloads\Documents\VEHICULAR EMMISSIONS\images\roboflow\train\_annotations.csv')
valid_annotations = pd.read_csv(r'C:\Users\bethg\Downloads\Documents\VEHICULAR EMMISSIONS\images\roboflow\valid\_annotations.csv')

# Train the first 300 images
train_annotations_subset = train_annotations.head(500)

# Set up training and testing directories
train_dir = r'C:\Users\bethg\Downloads\Documents\VEHICULAR EMMISSIONS\images\roboflow\train'
valid_dir = r'C:\Users\bethg\Downloads\Documents\VEHICULAR EMMISSIONS\images\roboflow\valid'

# Preprocess data (get all of the pixel values between 1 and 0 - scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Import data from directories and turn it into batches
train_data_subset = train_datagen.flow_from_dataframe(dataframe=train_annotations_subset,
                                                      directory=train_dir,
                                                      x_col='filename',
                                                      y_col='class',
                                                      batch_size=20,
                                                      target_size=(224, 224),
                                                      class_mode='binary',
                                                      seed=42)

valid_data = valid_datagen.flow_from_dataframe(dataframe=valid_annotations,
                                               directory=valid_dir,
                                               x_col='filename',
                                               y_col='class',
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode='binary',
                                               seed=42)
# Create a CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Create a TensorBoard callback
log_dir = r'C:\Users\bethg\Downloads\Desktop_prev\Untitled Folder\tensorboard' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Fit the model using the first 300 images and TensorBoard callback
history_1 = model_1.fit(train_data_subset,
                        epochs=25,
                        steps_per_epoch=len(train_data_subset),
                        validation_data=valid_data,
                        validation_steps=len(valid_data),
                        callbacks=[tensorboard_callback])

# # Save the trained model inside the folder
# save_folder = r'C:\Users\bethg\Downloads\Desktop_prev\Untitled Folder'
# os.makedirs(save_folder, exist_ok=True)
# model_1.save(os.path.join(save_folder, 'win11_CNN_model_.keras'))



Found 500 validated image filenames belonging to 2 classes.
Found 261 validated image filenames belonging to 2 classes.
Epoch 1/25
25/25 [==============================] - 39s 2s/step - loss: 0.4113 - accuracy: 0.8580 - val_loss: 2.1558 - val_accuracy: 0.1456
Epoch 2/25
25/25 [==============================] - 9s 356ms/step - loss: 0.3730 - accuracy: 0.8780 - val_loss: 1.5643 - val_accuracy: 0.1456
Epoch 3/25
25/25 [==============================] - 10s 382ms/step - loss: 0.3581 - accuracy: 0.8780 - val_loss: 1.8471 - val_accuracy: 0.1456
Epoch 4/25
25/25 [==============================] - 9s 376ms/step - loss: 0.3683 - accuracy: 0.8780 - val_loss: 2.2802 - val_accuracy: 0.1456
Epoch 5/25
25/25 [==============================] - 12s 470ms/step - loss: 0.3573 - accuracy: 0.8780 - val_loss: 2.1524 - val_accuracy: 0.1456
Epoch 6/25
25/25 [==============================] - 9s 378ms/step - loss: 0.2983 - accuracy: 0.8760 - val_loss: 2.3388 - val_accuracy: 0.1379
Epoch 7/25
25/25 [==========

# TESTING CODE


In [17]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the trained model
model_path = r'C:\Users\bethg\Downloads\Desktop_prev\Untitled Folder\win11_CNN_model_.keras'
model = load_model(model_path)

# Load the testing annotations
test_annotations = pd.read_csv(r'C:\Users\bethg\Downloads\Desktop_prev\Untitled Folder\roboflow\test\_annotations.csv')

# Load images from the directory
test_images_path = r'C:\Users\bethg\Downloads\Desktop_prev\Untitled Folder\roboflow\test'
test_images = []

# Loop through all files in the directory
for i, filename in enumerate(test_annotations['filename']):
    print(f"Loading image {i+1}/{len(test_annotations)}: {filename}")
    try:
        img = plt.imread(os.path.join(test_images_path, filename))
        if img is None:
            print(f"Error: Image not loaded for filename: {filename}")
        else:
            test_images.append(img)
    except Exception as e:
        print(f"Error loading image: {filename}. Exception: {e}")

if not test_images:
    print("No valid images loaded. Exiting.")
    exit()

print("Number of loaded images:", len(test_images))

# Convert list to NumPy array
test_images = np.stack(test_images, axis=0)

# Resize images to match model input shape
resized_images = []
for img in test_images:
    resized_img = cv2.resize(img, (224, 224))
    resized_images.append(resized_img)

test_images_resized = np.array(resized_images)

# Make predictions
predictions = model.predict(test_images_resized)

# Generate bounding boxes
for i, prediction in enumerate(predictions):
    filename = test_annotations['filename'][i]
    img = plt.imread(os.path.join(test_images_path, filename))

    if img is None:
        print(f"Error: Image not loaded for filename: {filename}")
        continue  # Skip this iteration

    xmax = test_annotations['xmax'][i]
    xmin = test_annotations['xmin'][i]
    ymax = test_annotations['ymax'][i]
    ymin = test_annotations['ymin'][i]

    # Convert the image to a UMat
    img = cv2.UMat(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    # Draw bounding box on image
    color = (0, 255, 0)  # Green color
    thickness = 2
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, thickness)

    # Display class label and confidence
    class_label = 'Smoky' if prediction >= 0.5 else 'Non-Smoky'
    confidence = f'{prediction[0]*100:.2f}%'
    text = f'{class_label} ({confidence})'
    org = (xmin, ymin - 10)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    cv2.putText(img, text, org, font, font_scale, color, thickness, cv2.LINE_AA)

    # Show image with bounding box
    print("Image shape:", img.get().shape)  # Add this line to check image shape
    cv2.imshow('Image with Classification Result', img.get())
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Loading image 1/124: img1.jpg
Loading image 2/124: img2.jpg
Loading image 3/124: img3.jpg
Loading image 4/124: img4.jpg
Loading image 5/124: img5.jpg
Loading image 6/124: img6.jpg
Loading image 7/124: img7.jpg
Loading image 8/124: img8.jpg
Loading image 9/124: img9.jpg
Loading image 10/124: img10.jpg
Loading image 11/124: img11.jpg
Loading image 12/124: img12.jpg
Loading image 13/124: img13.jpg
Loading image 14/124: img14.jpg
Loading image 15/124: img15.jpg
Loading image 16/124: img16.jpg
Loading image 17/124: img17.jpg
Loading image 18/124: img18.jpg
Loading image 19/124: img19.jpg
Loading image 20/124: img20.jpg
Loading image 21/124: img21.jpg
Loading image 22/124: img22.jpg
Loading image 23/124: img23.jpg
Loading image 24/124: img24.jpg
Loading image 25/124: img25.jpg
Loading image 26/124: img26.jpg
Loading image 27/124: img27.jpg
Loading image 28/124: img28.jpg
Loading image 29/124: img29.jpg
Loading image 30/124: img30.jpg
Loading image 31/124: img31.jpg
Loading image 32/124: img3

Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
Image shape: (640, 640, 3)
I

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bethg\\Downloads\\Desktop_prev\\Untitled Folder\\roboflow\\test\\img75.jpg'